In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(0, '/home/apatch/miniconda3/bin/tracktor/tracktor')
import tracktor as tr
import cv2
import sys
import scipy.signal
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist

## Global parameters
This cell (below) enlists user-defined parameters

In [2]:
# colours is a vector of BGR values which are used to identify individuals in the video
# since we only have one individual, the program will only use the first element from this array i.e. (0,0,255) - red
# number of elements in colours should be greater than n_inds (THIS IS NECESSARY FOR VISUALISATION ONLY)
n_inds = 5
colours = [(0,0,255),(0,255,255),(255,0,255),(255,255,255),(255,255,0),(255,0,0),(0,255,0),(0,0,0)]

# this is the block_size and offset used for adaptive thresholding (block_size should always be odd)
# these values are critical for tracking performance
block_size = 15
offset = 13

# the scaling parameter can be used to speed up tracking if video resolution is too high (use value 0-1)
scaling = 1.0

# minimum area and maximum area occupied by the animal in number of pixels
# this parameter is used to get rid of other objects in view that might be hard to threshold out but are differently sized
min_area = 20
max_area = 1000

# mot determines whether the tracker is being used in noisy conditions to track a single object or for multi-object
# using this will enable k-means clustering to force n_inds number of animals
mot = True
kmeans_all = False

# name of source video and paths
#video = 'test'
#video = '05SF_20m_30fps'
video = 'SFgroupOf5_t2_20scut'
home_path = '/data1/cavefish/experiment/'
input_vidpath = home_path + 'raw/CircleTank/20190116/' + video + '.avi'
output_vidpath = home_path +'output/' + video + '_tracked.avi'
output_filepath = home_path + 'output/' + video + '_tracked.dat'
codec = 'DIVX' # try other codecs if the default doesn't work ('DIVX', 'avc1', 'XVID') note: this list is non-exhaustive

The cell below runs the tracking code

In [3]:
## Open video
cap = cv2.VideoCapture(input_vidpath)
if cap.isOpened() == False:
    sys.exit('Video file cannot be read! Please check input_vidpath to ensure it is correctly pointing to the video file.')

## Video writer class to output video with contour and centroid of tracked object(s)
# make sure the frame size matches size of array 'final'
fourcc = cv2.VideoWriter_fourcc(*codec)
output_framesize = (int(cap.read()[1].shape[1]*scaling),int(cap.read()[1].shape[0]*scaling))
out = cv2.VideoWriter(filename = output_vidpath, fourcc = fourcc, fps = 30.0, frameSize = output_framesize, isColor = True)

## Individual location(s) measured in the last and current step
meas_last = list(np.zeros((n_inds,2)))
meas_now = list(np.zeros((n_inds,2)))

last = 0
df = []
frame_num = 0
fps = 30.

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    this = cap.get(1)
    if ret == True:
        frame = cv2.resize(frame, None, fx = scaling, fy = scaling, interpolation = cv2.INTER_LINEAR)
        thresh = tr.colour_to_thresh(frame, block_size, offset)
        final, contours, meas_last, meas_now = tr.detect_and_draw_contours(frame, thresh, meas_last, meas_now, min_area, max_area)
        if kmeans_all or ( mot and len(meas_now) != n_inds): 
            contours,meas_now = tr.apply_k_means(contours,n_inds,meas_now)
        row_ind, col_ind = tr.hungarian_algorithm(meas_last, meas_now)
        final, meas_now, df = tr.reorder_and_draw(final, colours, n_inds, col_ind, meas_now, df, mot, this)
        
        
        # Create output dataframe
        outputlist = []
        outputlist.append(len(meas_now))
        #distlist = cdist(meas_last,meas_now)
        for i in range(len(meas_now)):
            #outputlist.append(distlist[i])
            outputlist.append(meas_now[i][0])
            outputlist.append(meas_now[i][1])
            
            #df.append([this, meas_now[i][0], meas_now[i][1]])
        
        # Create line of output
        df.append(outputlist)
        
        # Display the resulting frame
        out.write(final)
        cv2.imshow('frame', final)
        if cv2.waitKey(1) == 27 or meas_now[0][0] < 20 or meas_now[0][0] > cap.get(3) - 20 or meas_now[0][1] < 20 or meas_now[0][1] > cap.get(4) - 20:
            break
            
    frame_num += 1
    if last >= this:
        break
    
    last = this

## Write positions to file
f = open(output_filepath,'w')
# write header
header = "#  frame  time  n_id"
for i in range(n_inds):
    header += "  pos" + str(i) + "_x  pos" + str(i) + "_y"
f.write(header + "\n")
# write data
for i in range(len(df)):
    line = " %7i %10.5f %i" % (i,float(i)/fps,df[i][0])
    for j in range(n_inds):
        if ( j >= (len(df[i])-1)/2 ):
            line += "    -     - "
        else:
            line += " %10.5f %10.5f" % (df[i][2*j+1],df[i][2*j+2])
    line += "\n"
    f.write(line)


## When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

SystemExit: Video file cannot be read! Please check input_vidpath to ensure it is correctly pointing to the video file.

/home/apatch/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
